In [168]:
import pandas as pd

### To do 
 
- Garder seulement les communes, ou traiter les différentes nomenclatures fonctionnelles (régions et dept)
- retraitement par fonction
- Script that does data dl, and sets up a sqlite database
- Visualisation with Dash
- Traitement intercommunalité
- Checker quels comptes ne sont pas traités
- Tests

In [303]:
import sys
from importlib import reload

import pandas as pd

import lib
reload(lib)
from lib import Cleaner, Compta, flag_vote_par_nature
import models

In [294]:
DATAFILE = 'BalanceSPL_Fonction_2013_Juin2019.csv'
EXPORT_FILE_NAME = 'bilan_2013.csv'

In [295]:
DATADIR = 'data_input/'
CATEGORIES = ['Commune']

In [297]:
# create dataframe from csv
df = pd.read_csv(
    filepath_or_buffer=DATADIR+DATAFILE,
    sep=';',
    encoding = "ISO-8859-1",
    dtype=str,
    na_filter=False,
    nrows=200000,
)

In [270]:
toulouse = raw_df[raw_df['IDENT'] == '21310555400017']
montpellier = raw_df[raw_df['IDENT'] == '21340172201787']

In [274]:
df = toulouse.copy()

In [298]:
cleaner = Cleaner({
    'EXER': int,
    'INSEE': Cleaner.insee,
    'FONCTION': Cleaner.fonction,
    'NDEPT': Cleaner.ndept,
    'OOBDEB': Cleaner.chiffre,
    'OOBCRE': Cleaner.chiffre,
    'SD': Cleaner.chiffre,
    'SC': Cleaner.chiffre,
    'OBNETDEB': Cleaner.chiffre,
    'OBNETCRE': Cleaner.chiffre,
})
df = cleaner.clean(df)

In [312]:
# Filter par catégorie
print(df.shape)
df = df[df['CATEG'].isin(CATEGORIES)]
print(df.shape)

(119754, 29)
(119754, 29)


In [324]:
import lib
reload(lib)
import compta 
reload(compta)
from lib import Cleaner, Compta, flag_vote_par_nature

In [314]:
df.loc[:, 'code_aggregat_compte'] = df['COMPTE'].apply(Compta.creer_aggregat_compte)

In [317]:
# Label aggregat de compte
df.loc[:, 'code_aggregat_compte'] = df['COMPTE'].apply(Compta.creer_aggregat_compte)
df.dropna(subset=['code_aggregat_compte'], inplace=True)
print(df.shape)

(87723, 31)


In [320]:
df['code_aggregat_compte'].apply(Compta.label_aggregat_compte)

987       11
988       11
993        7
994        7
995        8
          ..
190570     1
190571     1
190572     1
190573     1
190574     1
Name: code_aggregat_compte, Length: 87723, dtype: int64

In [330]:
reload(lib)

<module 'lib' from '/Users/gregoiredavid/Code/depenses_loc/lib.py'>

In [331]:
df.loc[:, 'aggregat_compte'] = df['code_aggregat_compte'].apply(Compta.label_aggregat_compte)

KeyError: 'label'

In [280]:
reload(lib)
from lib import Compta

In [281]:
import compta 
reload(compta)

<module 'compta' from '/Users/gregoiredavid/Code/depenses_loc/compta.py'>

In [282]:
df = Compta.calcul_depenses_par_aggregat_compte(df)

In [283]:
cols = ['code_aggregat_compte', 'aggregat_compte', 'OBNETCRE',
        'OBNETDEB','OBNETCRE','OOBDEB','OOBCRE','SD','SC', 'calcul_depense', 'depense']
df.loc[df['code_aggregat_compte'] == 10, cols]

,code_aggregat_compte,aggregat_compte,OBNETCRE,OBNETDEB,OBNETCRE,OOBDEB,OOBCRE,SD,SC,calcul_depense,depense
1103873,10.0,Prets accordes,0.00,0.00,0.00,0.0,0.0,3470671.67,0.00,OBNETDEB,0.00
1103875,10.0,Prets accordes,0.00,0.00,0.00,0.0,0.0,16226888.52,0.00,OBNETDEB,0.00
1103876,10.0,Prets accordes,0.00,0.00,0.00,0.0,0.0,188433.21,0.00,OBNETDEB,0.00
1103877,10.0,Prets accordes,0.00,0.00,0.00,0.0,0.0,16400.07,0.00,OBNETDEB,0.00
1104051,10.0,Prets accordes,2381739.46,1179921.57,2381739.46,0.0,0.0,0.00,1201817.89,OBNETDEB,1179921.57
1104052,10.0,Prets accordes,0.00,5002.46,0.00,0.0,0.0,5002.46,0.00,OBNETDEB,5002.46
1104053,10.0,Prets accordes,3600.00,305000.00,3600.00,0.0,0.0,301400.00,0.00,OBNETDEB,305000.00
1105217,10.0,Prets accordes,0.00,0.00,0.00,0.0,0.0,0.00,0.00,OBNETDEB,0.00


In [284]:
# Aggréger code fonctions
df['code_aggregat_fonction'] = df['FONCTION'].apply(Compta.creer_aggregat_fonction)

# Flag votes par nature et corriger l'aggrégat
flags = flag_vote_par_nature(df)
df = df.merge(flags, on='IDENT')

correction = Compta.corriger_aggregat_fonction_vote_par_nature
mask = df['vote_par_nature']
df.loc[mask, 'code_aggregat_fonction'] = df.loc[mask, 'FONCTION'].apply(correction)


In [285]:
df['aggregat_fonction'] = df['code_aggregat_fonction'].apply(Compta.label_aggregat_fonction)


In [289]:
grouped_df = (
    df
    .groupby([
        'siren',
        'NDEPT',
        'INSEE',
        'CTYPE',
        'NOMEN',
        'CATEG',
        'LBUDG',
        'IDENT',
        'code_aggregat_fonction',
        'aggregat_fonction',
        'code_aggregat_compte',
        'aggregat_compte',
    ])
    .agg({'depense': 'sum'})
    .reset_index()
)

In [290]:
# import et join pop data
grouped_df['code_commune'] = grouped_df['NDEPT'] + grouped_df['INSEE']
pop_data = models.population_data()
grouped_data_w_pop = grouped_df.merge(pop_data, how='left', on='code_commune')

In [292]:
grouped_data_w_pop.to_csv('toulouse_2018.csv')

In [333]:
bilan_2018 = pd.read_csv('data_output/bilan_2018.csv', dtype=str, na_filter=False)

In [390]:
communes

[{'label': 'Dunkerque', 'value': '59183'},
 {'label': 'Thizy-les-Bourgs', 'value': '69248'},
 {'label': 'Pont-de-Roide-Vermondans', 'value': '25463'},
 {'label': 'Vaugneray', 'value': '69255'},
 {'label': 'Tinchebray-Bocage', 'value': '61486'},
 {'label': 'Kaysersberg Vignoble', 'value': '68162'},
 {'label': 'Baugé-en-Anjou', 'value': '49018'},
 {'label': "Le Lion-d'Angers", 'value': '49176'},
 {'label': 'Epagny Metz-Tessy', 'value': '74112'},
 {'label': 'Chemillé-en-Anjou', 'value': '49092'},
 {'label': 'Beaupréau-en-Mauges', 'value': '49023'},
 {'label': 'Bornel', 'value': '60088'},
 {'label': 'Val-au-Perche', 'value': '61484'},
 {'label': 'Torigny-les-Villes', 'value': '50601'},
 {'label': 'Entrelacs', 'value': '73010'},
 {'label': 'Faverges-Seythenex', 'value': '74123'},
 {'label': 'Les Villages Vovéens', 'value': '28422'},
 {'label': 'Villeneuve-en-Retz', 'value': '44021'},
 {'label': "Ploeuc-L'Hermitage", 'value': '22203'},
 {'label': 'Essarts en Bocage', 'value': '85084'},
 {'la